In [1]:
from metavision_core.event_io import EventsIterator, load_events
import tonic
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import random
from IPython.utils import io
# import sdl2.ext

import models
import utils

In [2]:
# %matplotlib inline
%matplotlib ipympl
# %matplotlib tk
plt.rc('animation', html='jshtml')
plt.rcParams["image.cmap"] = "bone"

In [3]:
file_name = "1hzplane"

# in_data = load_events("./data/Line/" + file_name + ".dat")
# in_data = np.load("./data/Line/" + file_name + ".npy")
# in_frames = np.load("./data/Line/"+file_name+"_frames.npy")[0:100]

In [4]:
in_data = tonic.datasets.NMNIST(save_to="./data") # (34, 34, 2) 3
# in_data = tonic.datasets.POKERDVS(save_to="./data") # (35, 35, 2) 3
# in_data = tonic.datasets.DVSGesture(save_to="./data") # (128, 128, 2) 2
# in_data = tonic.datasets.ASLDVS(save_to="./data") # (240, 180, 2) 3

in_data.sensor_size

(34, 34, 2)

In [5]:
noise = tonic.transforms.UniformNoise(
    sensor_size=in_data.sensor_size[:-1]+(1,),
    n=int(4172),
)
# to_frame = tonic.transforms.ToFrame(in_data.sensor_size[:-1]+(1,), time_window=100)
to_frame = tonic.transforms.ToFrame(in_data.sensor_size[:-1]+(1,), n_time_bins=100)

In [6]:
events, label = in_data[random.randint(0, len(in_data)-1)]
events = events.squeeze()
events = np.array(list(filter(lambda ev: ev[3]==0, events)))
# print(events)
print(label)

true_frames = torch.tensor(to_frame(events)).float()#.cuda(0)
in_frames = torch.tensor(to_frame(noise(events))).float()#.cuda(0)

0


In [7]:
# ani.save("./img/nmnist-9.gif", writer="imagemagick")

In [8]:
net = models.Fast1()#.cuda(0)
with torch.inference_mode():
    out_frames = net(in_frames)

In [9]:
in_frames = utils.frame_merge(in_frames,5)
out_frames = utils.frame_merge(out_frames,5)
true_frames = utils.frame_merge(true_frames,5)

In [10]:
loss=nn.L1Loss()
print("noisy loss:"+ str(loss(in_frames,true_frames)))
print("denoised loss:"+ str(loss(out_frames,true_frames)))

noisy loss:tensor(0.1804)
denoised loss:tensor(0.2189)


In [11]:
in_frames = in_frames.cpu()
out_frames = out_frames.cpu()
true_frames = true_frames.cpu()

In [12]:
with io.capture_output() as captured:
    anim = tonic.utils.plot_animation(np.concatenate((in_frames,out_frames,true_frames),axis=3), 30)
    # anim = tonic.utils.plot_animation(np.concatenate((in_frames,out_frames),axis=3), 300)
display(anim)

In [13]:
# anim.save("./img/dataline3.gif", fps=300)